Andreas Bjelland Berg *(ID: 767921)*, Jitka Polaskova *(ID: 566613)* and Andreas Brennsæter *(ID: 507400)*

**Kaggle competition name:** Moscow housing

**Kaggle team name:** Team 95

In [149]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

from catboost import CatBoostRegressor, Pool

pd.set_option('display.max_rows', None)

%matplotlib inline

# Read files
We read the raw kaggle-given file from GitHub, to simplify running the notebook. These files have **not** been preprocessed, and are the same as what's given from Kaggle.

In [150]:
def read_file(url):
  url = url + "?raw=true"
  df = pd.read_csv(url)
  return df

url = "https://github.com/andbren/TDT-4173/blob/main/dataset/apartments_train.csv"
apartments = read_file(url)

url = "https://github.com/andbren/TDT-4173/blob/main/dataset/buildings_train.csv"
buildings = read_file(url)

print(f'All apartments have an associated building: {apartments.building_id.isin(buildings.id).all()}')
data = pd.merge(apartments, buildings.set_index('id'), how='left', left_on='building_id', right_index=True)

All apartments have an associated building: True


In [151]:
url = "https://github.com/andbren/TDT-4173/blob/main/dataset/apartments_test.csv"
apartments_test = read_file(url)

url = "https://github.com/andbren/TDT-4173/blob/main/dataset/buildings_test.csv"
buildings_test = read_file(url)

print(f'All apartments have an associated building: {apartments.building_id.isin(buildings.id).all()}')
data_test = pd.merge(apartments_test, buildings_test.set_index('id'), how='left', left_on='building_id', right_index=True)

All apartments have an associated building: True


# Define target functions
RMSLE is missing from a lot of ML-libraries, so we define it ourself.

In [152]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

def evaluate_predictions(predictions: pd.DataFrame, y_true: pd.DataFrame):
    """Evaluate predictions, the same way as done when uploading to Kaggle.

    Args:
      predictions: pandas DataFrame with predictions. Should be in the same
        order as the True data.
    
    Example:
      >>> # model = a previously trained model
      >>> results = model.predict(X_valid)
      >>> score = evaluate_predictions(results, y_valid)
    """
    return root_mean_squared_log_error(np.expm1(y_true), np.expm1(predictions))

# Preprocessing

## Fill in missing values

In [153]:
default_values = {   
    "seller":             4,        # Add a new category to seller - UNKNOWN = 4
    "area_kitchen":       np.median(data["area_kitchen"].dropna()),
    "area_living":        np.median(data["area_living"].dropna()),
    "layout":             3,        # Add a new category to layout - UNKNOWN = 3
    "ceiling":            np.median(data["ceiling"].dropna()),
    "bathrooms_shared":   0,
    "bathrooms_private":  0,
    "windows_court":      2,        # Change "windows_court" to categorical. New category - UNKNOWN = 2
    "windows_street":     2,        # Change "windows_street" to categorical. New category - UNKNOWN = 2
    "balconies":          0,
    "loggias":            0,
    "condition":          4,        # Add a new category to condition - UNKNOWN = 4
    "phones":             0,
    "new":                2,        # Change "new" to be categorical. New category - UNKNOWN = 2
    "district":           12,       # Add new category to district - UNKNOWN = 12
    "constructed":        np.median(data["constructed"].dropna()),
    "material":           7,        # Add new category to material - UNKNOWN = 7
    "elevator_without":   0,
    "elevator_passenger": 0,
    "elevator_service":   0,
    "parking":            3,         # Add new category to parking - UNKNOWN = 3
    "garbage_chute":      0,
    "heating":            4,         # Add new category to heating - UNKNOWN = 4
}

data.fillna(value=default_values, inplace = True)
data_test.fillna(value=default_values, inplace = True)

## Drop duplicates
(Not done in CSV that gives good results)

In [154]:
# print(f"Duplicated rows in data: ( {data.duplicated().sum()} ) of ( {data.shape[0]} )")
# data.drop_duplicates(inplace=True)
# print(f"Duplicated rows in data after removal: ( {data.duplicated().sum()} ) of ( {data.shape[0]} )")

## Remove outliers in data

In [155]:
_rows = data.shape[0]
print(f"Data rows before removing outliers: ( {_rows} )")

data.drop(data[data["price"] > 1.5e9].index, inplace=True)
data.drop(data[(data["price"] > 0.5e9) & (data["seller"] == 1)].index, inplace=True)
data.drop(data[data["area_living"] > 600].index, inplace=True)
data.drop(data[(data["price"] > 0.5e9) & (data["constructed"] > 1900) & (data["constructed"] < 1925)].index, inplace=True,)

print(f"Data rows after removing outliers: ( {data.shape[0]} ), removed ( {_rows - data.shape[0]} ) rows")

Data rows before removing outliers: ( 23285 )
Data rows after removing outliers: ( 23277 ), removed ( 8 ) rows


## Transform latitude/longitude-outliers in test data

In [156]:
# Transform extreme outliers in data_test
print(
    "Test rows where latitude < 50 || longitude < 30 || longitude > 40: (",
    data_test[(data_test['latitude'] < 50) | (data_test['longitude'] < 30) | (data_test['longitude'] > 40)].shape[0], 
    ") of total rows ( ", data_test.shape[0], ")"
)
data_test.loc[data_test["latitude"] < 50, "latitude"] = data_test[data_test["latitude"] >= 50]["latitude"].min()
data_test.loc[data_test["longitude"] < 30, "longitude"] = data_test[data_test["longitude"] >= 30]["longitude"].min()
data_test.loc[data_test["longitude"] > 40, "longitude"] = data_test[data_test["longitude"] <= 40]["longitude"].max()
print(
    "Test rows where latitude < 50 || longitude < 30 || longitude > 40 after transform: (",
    data_test[(data_test['latitude'] < 50) | (data_test['longitude'] < 30) | (data_test['longitude'] > 40)].shape[0], 
    ") of total rows ( ", data_test.shape[0], ")"
)

Test rows where latitude < 50 || longitude < 30 || longitude > 40: ( 7 ) of total rows (  9937 )
Test rows where latitude < 50 || longitude < 30 || longitude > 40 after transform: ( 0 ) of total rows (  9937 )


## Transform ceiling errors, likely from input errors

In [157]:
print(f"Data rows where ceiling > 10: ( {data[data['ceiling'] > 10].shape[0]} ) of total rows ( {data.shape[0]} )")
data.loc[data["ceiling"] > 100, "ceiling"] /= 10
data.loc[data["ceiling"] > 10, "ceiling"] /= 10
print(f"Data rows where ceiling > 10 after transform: ( {data[data['ceiling'] > 10].shape[0]} ) of total rows ( {data.shape[0]} )")

Data rows where ceiling > 10: ( 23 ) of total rows ( 23277 )
Data rows where ceiling > 10 after transform: ( 0 ) of total rows ( 23277 )


In [158]:
print(f"Test rows where ceiling > 10: ( {data_test[data_test['ceiling'] > 10].shape[0]} ) of total rows ( {data_test.shape[0]} )")
data_test.loc[data_test["ceiling"] > 100, "ceiling"] /= 10
data_test.loc[data_test["ceiling"] > 10, "ceiling"] /= 10
print(f"Test rows where ceiling > 10 after transform: ( {data_test[data_test['ceiling'] > 10].shape[0]} ) of total rows ( {data_test.shape[0]} )")

Test rows where ceiling > 10: ( 16 ) of total rows ( 9937 )
Test rows where ceiling > 10 after transform: ( 0 ) of total rows ( 9937 )


## Fix apartments where area_living + area_kitchen > area_total

In [159]:
# Set area_living and area_kitchen to be a ratio * area_total if their sum is higher than area_total
living_to_total_ratio = np.mean(data["area_living"].append(data_test["area_living"]) / data["area_total"].append(data_test["area_total"]))
kitchen_to_total_ratio = np.mean(data["area_kitchen"].append(data_test["area_kitchen"]) / data["area_total"].append(data_test["area_total"]))

In [160]:
print(
    "Data rows where area_living + area_kitchen > area_total: (",
    data[(data["area_living"] + data["area_kitchen"]) > data["area_total"]].shape[0],
    ") of total rows: (", data.shape[0], ")"
)
data.loc[(data["area_living"] + data["area_kitchen"]) > data["area_total"], "area_kitchen"] = kitchen_to_total_ratio * data["area_total"]
data.loc[(data["area_living"] + data["area_kitchen"]) > data["area_total"], "area_living"] = living_to_total_ratio * data["area_total"]
print(
    "Data rows where area_living + area_kitchen > area_total after transform: (",
    data[(data["area_living"] + data["area_kitchen"]) > data["area_total"]].shape[0],
    ") of total rows: (", data.shape[0], ")"
)

Data rows where area_living + area_kitchen > area_total: ( 1095 ) of total rows: ( 23277 )
Data rows where area_living + area_kitchen > area_total after transform: ( 0 ) of total rows: ( 23277 )


In [161]:
print(
    "Test rows where area_living + area_kitchen > area_total: (",
    data_test[(data_test["area_living"] + data_test["area_kitchen"]) > data_test["area_total"]].shape[0],
    ") of total rows: (", data_test.shape[0], ")"
)
data_test.loc[(data_test["area_living"] + data_test["area_kitchen"]) > data_test["area_total"], "area_kitchen"] = kitchen_to_total_ratio * data_test["area_total"]
data_test.loc[(data_test["area_living"] + data_test["area_kitchen"]) > data_test["area_total"], "area_living"] = living_to_total_ratio * data_test["area_total"]
print(
    "Test rows where area_living + area_kitchen > area_total after transform: (",
    data_test[(data_test["area_living"] + data_test["area_kitchen"]) > data_test["area_total"]].shape[0],
    ") of total rows: (", data_test.shape[0], ")"
)

Test rows where area_living + area_kitchen > area_total: ( 538 ) of total rows: ( 9937 )
Test rows where area_living + area_kitchen > area_total after transform: ( 0 ) of total rows: ( 9937 )


## Fix apartments where floor > stories in building

In [162]:
print("Data rows where floor > stories: (", data[data["floor"] > data["stories"]].shape[0], ") of total rows: (", data.shape[0], ")")
data.loc[data["floor"] > data["stories"], "floor"] = data["stories"]
print("Data rows where floor > stories after transform: (", data[data["floor"] > data["stories"]].shape[0], ")")

Data rows where floor > stories: ( 321 ) of total rows: ( 23277 )
Data rows where floor > stories after transform: ( 0 )


In [163]:
print("Test rows where floor > stories: (", data_test[data_test["floor"] > data_test["stories"]].shape[0], ") of total rows: (", data_test.shape[0], ")")
data_test.loc[data_test["floor"] > data_test["stories"], "floor"] = data_test["stories"]
print("Test rows where floor > stories after transform: (", data_test[data_test["floor"] > data_test["stories"]].shape[0], ")")

Test rows where floor > stories: ( 93 ) of total rows: ( 9937 )
Test rows where floor > stories after transform: ( 0 )


## Fix missing features in data_test

In [164]:
data_test['latitude'] = data_test['latitude'].fillna(data_test['latitude'].median())
data_test['longitude'] = data_test['longitude'].fillna(data_test['longitude'].median())

# Add new features

## Add meta-column "price_bin"
price_bin is a categorization of price, used for weights in the polar coordinates and to stratify train/test-splitting

In [165]:
NUM_BUCKETS = 10
log_price = np.log10(data['price'])

price_bin_max = log_price.max()
price_bin_min = log_price.min()
price_bin_size = (price_bin_max - price_bin_min) / NUM_BUCKETS

price_bins = [
    i*price_bin_size + price_bin_min for i in range(NUM_BUCKETS)
]
labels = [i for i in range(len(price_bins) - 1)]

data['price_bin'] = pd.cut(log_price, bins=price_bins, labels=labels)
data["price_bin"].fillna(8, inplace=True)

## Add polar coordinates

In [166]:
# Convert latitude, longitude to polar coordinates
def cartesian_to_polar_coordinates(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)
    
geographical_weighted_center_latitude = np.average(data["latitude"], weights=data["price_bin"])
geographical_weighted_center_longitude = np.average(data["longitude"], weights=data["price_bin"])

In [167]:
delta_latitude = data["latitude"] - geographical_weighted_center_latitude
delta_longitude = data["longitude"] - geographical_weighted_center_longitude

data["distance_from_center"], data["angle"] = cartesian_to_polar_coordinates(delta_latitude, delta_longitude)

In [168]:
delta_latitude = data_test["latitude"] - geographical_weighted_center_latitude
delta_longitude = data_test["longitude"] - geographical_weighted_center_longitude

data_test["distance_from_center"], data_test["angle"] = cartesian_to_polar_coordinates(delta_latitude, delta_longitude)

## Add average_room_size

In [169]:
data["average_room_size"] = data["area_total"] / data["rooms"]
data_test["average_room_size"] = data_test["area_total"] / data_test["rooms"]

## Add bathroom_amount

In [170]:
data["bathroom_amount"] = data["bathrooms_private"] + data["bathrooms_shared"]
data_test["bathroom_amount"] = data_test["bathrooms_private"] + data_test["bathrooms_shared"]

## Add relative floor (how high up the building the apartment is)

In [171]:
data["relative_floor"] = data["floor"] / data["stories"]
data_test["relative_floor"] = data_test["floor"] / data_test["stories"]

# Drop columns

In [172]:
# layout has lots of missing values - drop the column from data and data_test
# address and street have string values, so we lose them as well
data.drop(["layout", "address", "street"], axis=1, inplace=True)
data_test.drop(["layout", "address", "street"], axis=1, inplace=True)

# Data conversions

In [173]:
needed_dtypes = {
    "seller": CategoricalDtype(categories=[0, 1, 2, 3, 4]),
    "floor": "uint8",
    "rooms": "uint8",
    "bathrooms_shared": "uint8",
    "bathrooms_private": "uint8",
    "windows_court": CategoricalDtype(categories=[0, 1, 2]),
    "windows_street": CategoricalDtype(categories=[0, 1, 2]),
    "balconies": "uint8",
    "loggias": "uint8",
    "condition": CategoricalDtype(categories=[0, 1, 2, 3, 4]),
    "phones": "uint8",
    "new": CategoricalDtype(categories=[0, 1, 2]),
    "district": CategoricalDtype(categories=list(range(13))),
    "constructed": "uint16",
    "material": CategoricalDtype(categories=list(range(8))),
    "stories": "uint8",
    "elevator_without": "bool",
    "elevator_passenger": "bool",
    "elevator_service": "bool",
    "parking": CategoricalDtype(categories=[0, 1, 2, 3]),
    "garbage_chute": "bool",
    "heating": CategoricalDtype(categories=[0, 1, 2, 3, 4]),
    "bathroom_amount": CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8]),
}
data = data.astype(needed_dtypes)
data_test = data_test.astype(needed_dtypes)

Save IDs separately and drop from datasets

In [174]:
test_Id = data_test['id']
data_test.drop('id', axis = 1, inplace = True)
data.drop('id', axis = 1, inplace = True)

# Train/test-split

In [175]:
data_train, data_valid = train_test_split(
    data.drop(["price_bin"], axis=1),
    test_size = 0.3,
    stratify = data["price_bin"],
    random_state = 42,
)

We are going to fit the model on a full train set. This is normally done only after first fitting on subset on train set and validating the results using validation set.

However, here we show the final version of the model and we already know which hyperparameter combination leads to the best result on train+validation set. Therefore we use that knowledge to fit directly on full set.

Alternatively, it is possible to skip the previous preprocessing steps and instead of it directly load preprocessed files, that were uploaded on GitHub for more convenience. The preprocessing steps of the data is identical to the steps shown here:

https://github.com/andbren/TDT-4173/blob/main/15.11/data_train%20(1).csv

https://github.com/andbren/TDT-4173/blob/main/15.11/data_valid%20(1).csv

https://github.com/andbren/TDT-4173/blob/main/15.11/test%20(2).csv

Logging skewed features

In [176]:
data_train["price"] = np.log1p(data_train["price"])
data_valid["price"] = np.log1p(data_valid["price"])

In [177]:
data_train["area_total"] = np.log1p(data_train["area_total"])
data_valid["area_total"] = np.log1p(data_valid["area_total"])
data_test["area_total"] = np.log1p(data_test["area_total"])

data_train["area_kitchen"] = np.log1p(data_train["area_kitchen"])
data_valid["area_kitchen"] = np.log1p(data_valid["area_kitchen"])
data_test["area_kitchen"] = np.log1p(data_test["area_kitchen"])

data_train["area_living"] = np.log1p(data_train["area_living"])
data_valid["area_living"] = np.log1p(data_valid["area_living"])
data_test["area_living"] = np.log1p(data_test["area_living"])

data_train.drop('loggias', axis = 1, inplace = True)
data_valid.drop('loggias', axis = 1, inplace = True)
data_test.drop('loggias', axis = 1, inplace = True)

data_train.drop('balconies', axis = 1, inplace = True)
data_valid.drop('balconies', axis = 1, inplace = True)
data_test.drop('balconies', axis = 1, inplace = True)


In [178]:
data_combined = data_train.append(data_valid, ignore_index = True)

# Model

In [179]:
categorical_columns = [
    "seller",
    "rooms",
    "bathrooms_shared",
    "bathrooms_private",
    "windows_court",
    "windows_street",
    "condition",
    "phones",
    "new",
    "district",
    "material",
    "elevator_without",
    "elevator_passenger",
    "elevator_service",
    "parking",
    "garbage_chute",
    "heating",
    "bathroom_amount",
]

In [180]:
X_test = data_test

y_train = data_train["price"]
y_valid = data_valid["price"]
y_combined = data_combined["price"]

X_train = data_train.drop("price", axis=1)
X_valid = data_valid.drop("price", axis=1)
X_combined = data_combined.drop("price", axis=1)

In [181]:
valid_set = Pool(
    X_valid,
    label = y_valid,
    cat_features = categorical_columns
)

In [186]:
half_model = CatBoostRegressor(
    cat_features = categorical_columns,
    custom_metric="MSLE",
    eval_metric="MSLE",
    border_count=254,
    ctr_target_border_count=2,
    depth=10,
    iterations=5000,
    l2_leaf_reg=1,
    one_hot_max_size=8,
)
half_model.fit(
    X_train,
    y_train,
    eval_set=valid_set,
    use_best_model=True,
    early_stopping_rounds=100,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0022009	test: 0.0021950	best: 0.0021950 (0)	total: 18.1ms	remaining: 1m 30s
1:	learn: 0.0020935	test: 0.0020898	best: 0.0020898 (1)	total: 37.5ms	remaining: 1m 33s
2:	learn: 0.0019918	test: 0.0019888	best: 0.0019888 (2)	total: 54.3ms	remaining: 1m 30s
3:	learn: 0.0018947	test: 0.0018925	best: 0.0018925 (3)	total: 71.7ms	remaining: 1m 29s
4:	learn: 0.0018018	test: 0.0017997	best: 0.0017997 (4)	total: 87.1ms	remaining: 1m 27s
5:	learn: 0.0017144	test: 0.0017139	best: 0.0017139 (5)	total: 104ms	remaining: 1m 26s
6:	learn: 0.0016311	test: 0.0016308	best: 0.0016308 (6)	total: 119ms	remaining: 1m 24s
7:	learn: 0.0015517	test: 0.0015527	best: 0.0015527 (7)	total: 133ms	remaining: 1m 23s
8:	learn: 0.0014770	test: 0.0014783	best: 0.0014783 (8)	total: 147ms	remaining: 1m 21s
9:	learn: 0.0014065	test: 0.0014080	best: 0.0014080 (9)	total: 161ms	remaining: 1m 20s
10:	learn: 0.0013406	test: 0.0013422	best: 0.0013422 (10)	total: 177ms	remaining: 1m 20s
11:	learn: 0.0012778	test: 0.0012811

Based on this graph, we can see that the validation flattens out around 100 iterations. We therefore select iterations = 100 as our target iterations.

In [183]:
half_model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'MSLE',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=2:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 1500,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'od_pval': 0,
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 8,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 1,
 'random_strength': 1,
 'od_type': 'Iter',
 'rsm': 1,
 'boost_from_average': True,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=2:TargetBord

In [194]:
params = half_model.get_params()
params["iterations"] = 100                  # See above
full_model = CatBoostRegressor(
    **params
)
full_model.fit(
    X_combined,
    y_combined
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0021934	total: 13.5ms	remaining: 2.01s
1:	learn: 0.0020834	total: 27.1ms	remaining: 2s
2:	learn: 0.0019785	total: 40.1ms	remaining: 1.97s
3:	learn: 0.0018789	total: 53.6ms	remaining: 1.96s
4:	learn: 0.0017874	total: 67.3ms	remaining: 1.95s
5:	learn: 0.0016993	total: 81ms	remaining: 1.95s
6:	learn: 0.0016164	total: 93.1ms	remaining: 1.9s
7:	learn: 0.0015382	total: 105ms	remaining: 1.87s
8:	learn: 0.0014637	total: 120ms	remaining: 1.88s
9:	learn: 0.0013924	total: 133ms	remaining: 1.86s
10:	learn: 0.0013271	total: 145ms	remaining: 1.83s
11:	learn: 0.0012641	total: 157ms	remaining: 1.8s
12:	learn: 0.0012067	total: 169ms	remaining: 1.78s
13:	learn: 0.0011504	total: 180ms	remaining: 1.75s
14:	learn: 0.0010986	total: 192ms	remaining: 1.73s
15:	learn: 0.0010492	total: 203ms	remaining: 1.7s
16:	learn: 0.0010028	total: 215ms	remaining: 1.68s
17:	learn: 0.0009577	total: 227ms	remaining: 1.66s
18:	learn: 0.0009163	total: 240ms	remaining: 1.65s
19:	learn: 0.0008771	total: 252ms	remainin

In [192]:
full_model_predictions = full_model.predict(X_test)
full_model_predictions = np.expm1(full_model_predictions)

In [193]:
submission = pd.DataFrame(data= {'id' : test_Id, 'price_prediction': full_model_predictions})
submission.to_csv('catboost_short_notebook_submission.csv', index=False)